In [1]:
import config
import models
import tensorflow as tf
import numpy as np
import os


In [2]:
train_data_path = "./benchmarks/FB15K/"
test_data_path = "./benchmarks/FB15K_OOV/"

train_file_path = "./res/model.vec.tf"
train_embedding_path = "./res/embedding.vec.json"

test_file_path = "./res/model_new.vec.tf"
test_embedding_path = "./res/embedding_new.vec.json"

# Run TransE To Create initial embeddings

In [ ]:
"""
Method:

Run the normal transe example (example_train_transe.py)
Write the embeddings as a file that can be read
use embeddings to initialize embedding layer in TransE_freeze.py

Append random embeddings for new entities and relations
set config freeze_train_embeddings = true
figure out how to update only the embeddings for a certain set of indices
figure out how to make sure that we only see examples using new items to speed convergence
compare new+old embeddings
"""
os.environ['CUDA_VISIBLE_DEVICES']='7'
con = config.Config()
#True: Input test files from the same folder.
con.set_in_path(train_data_path)
con.set_test_link_prediction(True)
con.set_test_triple_classification(True)
con.set_work_threads(8)
con.set_train_times(10)
con.set_nbatches(20)
con.set_alpha(0.001)
con.set_margin(1.0)
con.set_bern(0)
con.set_dimension(100)
con.set_ent_neg_rate(1)
con.set_rel_neg_rate(0)
con.set_opt_method("SGD")

In [ ]:
#Models will be exported via tf.Saver() automatically.
con.set_export_files(train_file_path, 0)
#Model parameters will be exported to json files automatically.
con.set_out_files(train_embedding_path)
#Initialize experimental settings.
con.init()

In [ ]:

#Set the knowledge embedding model
con.set_model(models.TransE)


In [ ]:
#Train the model.
con.run()

# Run TransE Freeze using Embeddings produced in above step

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='7'
#Input training files from benchmarks/FB15K/ folder.
con = config.Config()
#True: Input test files from the same folder.
con.set_in_path(test_data_path)
con.set_test_link_prediction(False)
con.set_test_triple_classification(False)
con.set_work_threads(8)
con.set_train_times(10)
con.set_nbatches(2) # Total obs = 18 . Need ~3 per batch, so batch size = total / n_batches 
con.set_alpha(0.001)
con.set_margin(1.0)
con.set_bern(0)
con.set_dimension(100)
con.set_ent_neg_rate(1)
con.set_rel_neg_rate(0)
con.set_opt_method("SGD")


con.set_freeze_train_embeddings(True)
con.set_embedding_initializer_path(train_embedding_path)


In [4]:
#Models will be exported via tf.Saver() automatically.
con.set_export_files(test_file_path, 0)
#Model parameters will be exported to json files automatically.
con.set_out_files(test_embedding_path)
#Initialize experimental settings.
con.init()
#Set the knowledge embedding model
con.set_model(models.TransE_freeze)

Too many entities!
Total Entities in data: 14968 
Total Entities in Embedding file: 14951


Too many relationships!
Total Relationships in data: 1352 
Total Relationships in Embedding file: 1345

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Pos_h
Tensor("model/input/transpose:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup/Identity:0", shape=(9, 1, 100), dtype=float32)
pos_t
Tensor("model/input/transpose:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup_1/Identity:0", shape=(9, 1, 100), dtype=float32)
pos_r
Tensor("model/input/transpose_2:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup_2/Identity:0", shape=(9, 1, 100), dtype=float32)
neg_h
Tensor("model/input/transpose_3:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup_3/Identity:0", shape=(9, 1, 100), dtype=float32)
neg_t
Tensor("model/input/transpose_3:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup_4/Identity:0", shape=(9, 1, 100), dtype=float32)
neg_r
Tensor("model/input/transpose_5:0", shape=(9, 1), dtype=int64)
Tensor("model/loss/embedding_lookup_5/Identity:0", shape=(9, 1, 100), dtype=float32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [5]:
#Train the model.
con.run()

Epoch: 0, loss: 9.79340934753418, time: 0.0
Epoch: 1, loss: 7.868074178695679, time: 0.0
Epoch: 2, loss: 8.77539348602295, time: 0.0
Epoch: 3, loss: 8.584557294845581, time: 9.5367431640625e-07
Epoch: 4, loss: 7.646888256072998, time: 1.1920928955078125e-06
Epoch: 5, loss: 7.066133499145508, time: 0.0
Epoch: 6, loss: 7.60280966758728, time: 0.0
Epoch: 7, loss: 3.622953176498413, time: 0.0
Epoch: 8, loss: 4.594324827194214, time: 9.5367431640625e-07
Epoch: 9, loss: 8.525736570358276, time: 9.5367431640625e-07


# Compare new and old embeddings

In [6]:
import json
with open("./res/embedding.vec.json", "r") as f: 
    old_embeddings = json.loads(f.read())
    old_ent_embeddings = old_embeddings["ent_embeddings"]
    old_rel_embeddings = old_embeddings["rel_embeddings"]


In [7]:
with open("./res/embedding_new.vec.json", "r") as f: 
    new_embeddings = json.loads(f.read())
    new_ent_embeddings = new_embeddings["ent_embeddings"]
    new_rel_embeddings = new_embeddings["rel_embeddings"]

In [11]:
old_ent_embeddings[1235][0:10] == new_ent_embeddings[1235][0:10]

True

In [ ]:
new_ent_embeddings[1233][0:10]

# Does anything actually happen to the embeddings?

In [ ]:
old_ent_embeddings[-1] == new_ent_embeddings[-1]

In [ ]:
# Should be false
old_rel_embeddings[-1] == new_rel_embeddings[-1]

In [ ]:
# Should be TRUE
old_rel_embeddings[-1] == new_rel_embeddings[-2]

In [ ]:
len(new_rel_embeddings)

In [ ]:
len(old_rel_embeddings)

In [ ]:
import numpy as np
np.concatenate((25*np.ones([3,1], dtype=np.int),50*np.ones([3,1], dtype=np.int)), 0) > 26